# SimPy Example for IE 306.02
SimPy is a powerful discrete-event simulation library in Python that can be used to model and simulate various systems, including queuing systems. 

This example shows how a simple M/M/1 system (1 server with an infinite capacity queue, exponential interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at random times. There is only a single operator in this call center, and picks up the first call waiting when she is available. 

We start by importing the necessary packages:

In [9]:
import simpy
import random

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, total number of customers that will depart during the run of the simulation, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. service rate) distribution.

In [10]:
RANDOM_SEED = 978
NUMBER_OF_CUSTOMERS = 100
INTERARRIVAL_RATE = 0.1 
SERVICE_RATE = 0.15
random.seed(RANDOM_SEED)

Define the necessary lists for bookkeeping:

In [11]:
customers = []#List of customers
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
queue_w_times = [] #Time spent by a customer while it waits for the operator (Queue waiting time Wq)

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to the service time distribution.

In [12]:
class Customer(object):
    def __init__(self, name, env, operator):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question())
            print('%s is done at %g' % (self.name, self.env.now))
            
    def ask_question(self):
        duration = random.expovariate(SERVICE_RATE)
        yield self.env.timeout(duration)
        service_times.append(duration)
        
        

Create customers that will join the queue during the run of the simulation:

In [13]:
def customer_generator(env, operator):
    """Generate new customers that call the line."""
    for i in range(NUMBER_OF_CUSTOMERS):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Cust %s' %(i+1), env, operator)
        customers.append(customer)  

Finally, create the main environment, resource and process, then run the simulation:

In [14]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator))
env.run() 

Cust 1 initiated a call at 44.1894
Cust 1 is assigned to an operator at 44.1894
Cust 1 is done at 44.84
Cust 2 initiated a call at 45.8734
Cust 2 is assigned to an operator at 45.8734
Cust 2 is done at 50.9534
Cust 3 initiated a call at 62.0814
Cust 3 is assigned to an operator at 62.0814
Cust 4 initiated a call at 67.8025
Cust 3 is done at 73.0814
Cust 4 is assigned to an operator at 73.0814
Cust 4 is done at 79.3449
Cust 5 initiated a call at 85.3643
Cust 5 is assigned to an operator at 85.3643
Cust 5 is done at 85.8552
Cust 6 initiated a call at 91.7531
Cust 6 is assigned to an operator at 91.7531
Cust 6 is done at 93.1381
Cust 7 initiated a call at 111.619
Cust 7 is assigned to an operator at 111.619
Cust 7 is done at 113.659
Cust 8 initiated a call at 135.158
Cust 8 is assigned to an operator at 135.158
Cust 9 initiated a call at 139.678
Cust 8 is done at 146.591
Cust 9 is assigned to an operator at 146.591
Cust 10 initiated a call at 152.726
Cust 11 initiated a call at 166.736
Cu

In [15]:
print(queue_w_times, "\n")

[0.0, 0.0, 0.0, 5.27888315315829, 0.0, 0.0, 0.0, 0.0, 6.913210827805074, 18.20529200814039, 17.17165901715427, 1.5481793110676279, 3.872131192126176, 6.894150136756309, 0.0, 0.0, 0.0, 5.0562965197251515, 15.712607949599999, 14.391498448543302, 0.0, 0.0, 0.0, 0.0, 5.841440437635299, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.45958639033205, 9.745378494989097, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.02443531750555, 9.788217138062691, 19.319568636140957, 22.12409204765777, 21.05953834753052, 11.445100860951811, 15.978819920699834, 0.0, 0.0, 2.317169933476862, 0.0, 1.4691078984624255, 0.6097845298586435, 15.205180469524521, 26.720979844374938, 9.061140285906845, 8.783699877470667, 5.728567133753472, 1.1904405352512413, 0.0, 0.0, 0.6459620860928226, 0.0, 0.28366577999895526, 46.068607959100405, 54.646604796856536, 54.08853988993553, 19.919176761569133, 4.071463605904569, 6.518487664049303, 0.0, 0.0, 0.0, 12.481845362304284, 17.117810312089773, 20.482394475537262, 27.31253519445056, 37.388800351386

In [16]:
print(service_times, "\n")

[0.6506461545072149, 5.0799347057934865, 11.000006878178091, 6.263497734288345, 0.4909178279439705, 1.3850796229977014, 2.040319869959479, 11.43317313989569, 24.34053517320568, 12.97565587888221, 3.2756204350936797, 4.916892721541834, 3.3985075273793064, 2.4081558744594598, 1.351999994113726, 5.321035207446573, 5.269397988118282, 11.72204465481192, 11.65355013378369, 0.6633602947822131, 6.804923394833477, 3.2626910691915447, 1.6129163006710119, 10.531882457383967, 2.1811369533278455, 2.3128334311547762, 0.158680228553818, 2.170646841467527, 1.6661766780676892, 0.49557948016287096, 2.8821196646126164, 16.65250145169647, 2.9333851029069553, 3.010879777209407, 1.4537095012945178, 1.1566226128530335, 1.1412031757843342, 4.433508926126945, 1.5730221600392094, 3.3246767970356683, 16.750576158690976, 0.9380219439875245, 9.777138715612232, 22.215148649356564, 3.161746651827123, 14.10982032413782, 8.2555991807363, 3.9250256558061825, 1.5104452329673004, 3.074297896511428, 1.540793197989955, 11.